In [1]:
import sympy
import numpy as np
from tqdm import tqdm
import dill
dill.settings["recurse"] = True
from sympy import Rational as R 

In [2]:
(
    reference_x,
    reference_y,
    reference_z,
    physical_x_0,
    physical_y_0,
    physical_z_0,
    physical_x_1,
    physical_y_1,
    physical_z_1,
    physical_x_2,
    physical_y_2,
    physical_z_2,
    physical_x_3,
    physical_y_3,
    physical_z_3,
) = sympy.symbols("x y z x_0 y_0 z_0 x_1 y_1 z_1 x_2 y_2 z_2 x_3 y_3 z_3")

f_interpolation = sympy.Matrix(sympy.symbols("f_0:4"))

In [3]:
v0 = {reference_x: 0, reference_y: 0, reference_z: 0}
v1 = {reference_x: 1, reference_y: 0, reference_z: 0}
v2 = {reference_x: 0, reference_y: 1, reference_z: 0}
v3 = {reference_x: 0, reference_y: 0, reference_z: 1}

# Get basis functions

In [4]:
monomial_basis = sympy.Matrix(
[
 1,
 reference_x,
 reference_y,
 reference_z,
]
)

In [5]:
Vander = sympy.Matrix(np.zeros((4, 4)))

for i, basis in enumerate(monomial_basis):
    
    Vander[i, 0] = basis.subs(v0)
    Vander[i, 1] = basis.subs(v1)
    Vander[i, 2] = basis.subs(v2)
    Vander[i, 3] = basis.subs(v3)

In [6]:
lagrange_p1_basis = Vander.inv()@monomial_basis

# Get mapping function

In [7]:
mapping_basis = Vander.inv() @ monomial_basis

global_x = (
      mapping_basis[0] * physical_x_0
    + mapping_basis[1] * physical_x_1
    + mapping_basis[2] * physical_x_2
    + mapping_basis[3] * physical_x_3
)
global_y = (
      mapping_basis[0] * physical_y_0
    + mapping_basis[1] * physical_y_1
    + mapping_basis[2] * physical_y_2
    + mapping_basis[3] * physical_y_3
)
global_z = (
      mapping_basis[0] * physical_z_0
    + mapping_basis[1] * physical_z_1
    + mapping_basis[2] * physical_z_2
    + mapping_basis[3] * physical_z_3
)

mapping_function = sympy.Matrix([global_x, global_y, global_z])

In [8]:
J_cofactor_T = mapping_function.jacobian([reference_x, reference_y, reference_z]).cofactor_matrix().T
J = J_cofactor_T / J_cofactor_T.det()

In [9]:
u = sympy.Matrix([sympy.Function("u")(reference_x, reference_y, reference_z)])
v = sympy.Matrix([sympy.Function("v")(reference_x, reference_y, reference_z)])

In [10]:
grad_u = u.jacobian([reference_x, reference_y, reference_z])@J_cofactor_T
grad_v = v.jacobian([reference_x, reference_y, reference_z])@J_cofactor_T

weak_form = grad_u@grad_v.T

In [13]:
N = 4
weak_form_functional = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = lagrange_p1_basis[idx]
    second = lagrange_p1_basis[jdx]

    A = weak_form.subs({u[0]:first, v[0]:second}).doit()
    B = first * second
    
    sym_int_A = sympy.integrate(A, (reference_z, 0, R(1) - reference_x - reference_y), (reference_y, 0, R(1) - reference_x), (reference_x, 0, R(1)))
    sym_int_B = sympy.integrate(B, (reference_z, 0, R(1) - reference_x - reference_y), (reference_y, 0, R(1) - reference_x), (reference_x, 0, R(1)))

    weak_form_functional[idx][jdx] = sym_int_A
    weak_form_right_part[idx][jdx] = sym_int_B
    

weak_form_functional = sympy.Matrix(weak_form_functional) / J_cofactor_T.det()**2
weak_form_right_part = sympy.Matrix(weak_form_right_part)

weak_form_functional = weak_form_functional * abs(J_cofactor_T.det())
weak_form_right_part = weak_form_right_part * abs(J_cofactor_T.det())

  0%|          | 0/16 [00:00<?, ?it/s]

100%|██████████| 16/16 [01:23<00:00,  5.22s/it]


In [14]:
weak_form_functional = sympy.Matrix(weak_form_functional)
weak_form_right_part = sympy.Matrix(weak_form_right_part)

f_interpolation = sympy.Matrix(f_interpolation)

In [15]:
lambdify_symbols = [    
    physical_x_0,
    physical_y_0,
    physical_z_0,
    physical_x_1,
    physical_y_1,
    physical_z_1,
    physical_x_2,
    physical_y_2,
    physical_z_2,
    physical_x_3,
    physical_y_3,
    physical_z_3,
]

In [16]:
weak_form_functional_lambdified = sympy.lambdify(lambdify_symbols, weak_form_functional, cse=True)

In [17]:
weak_form_right_part_lambdified = sympy.lambdify([*lambdify_symbols, *f_interpolation], weak_form_right_part@f_interpolation, cse=True)

In [19]:
dill.dump(weak_form_functional_lambdified, open("../calculations/3d_lagrange_linear_poisson_matrix", "wb"))
dill.dump(weak_form_right_part_lambdified, open("../calculations/3d_lagrange_linear_poisson_right_values", "wb"))